In [29]:
import psycopg2
from dotenv import load_dotenv
import os

# Load environment variables from .env file

# Get the current directory
current_dir = os.getcwd()

# Climb two directories up
for _ in range(2):
    current_dir = os.path.dirname(current_dir)

# Join the parent directory with the .env filename
dotenv_path = os.path.join(current_dir, '.env')

# Load the .env file
load_dotenv(dotenv_path)


# Define your connection parameters
db_name = os.getenv("POSTGRES_DB")
db_user = os.getenv("POSTGRES_USER")
db_password = os.getenv("POSTGRES_PASSWORD")
db_host = os.getenv("POSTGRES_HOST", "localhost")  # or the IP address of your Docker container
db_port = os.getenv("DB_PORT", "5432")  # or the port your PostgreSQL server is listening on

# Create a connection
conn = psycopg2.connect(
    dbname=db_name,
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)

# Create a cursor object
cur = conn.cursor()

# Execute a query
cur.execute("SELECT * FROM events ; ")

# Fetch all the rows
rows = cur.fetchall()

In [25]:
# Execute a query to get all schemas
cur.execute("SELECT schema_name FROM information_schema.schemata;")

# Fetch all the rows
schemas = cur.fetchall()

# Print all schemas
for schema in schemas:
    print(f"Schema: {schema[0]}")
    # Execute a query to get all tables in the current schema
    cur.execute(f"SELECT table_name FROM information_schema.tables WHERE table_schema = '{schema[0]}';")
    # Fetch all the rows
    tables = cur.fetchall()
    # Print all tables in the current schema
    for table in tables:
        print(f"  Table: {table[0]}")

Schema: public
  Table: events
Schema: information_schema
  Table: collations
  Table: information_schema_catalog_name
  Table: applicable_roles
  Table: domain_constraints
  Table: administrable_role_authorizations
  Table: collation_character_set_applicability
  Table: attributes
  Table: character_sets
  Table: column_udt_usage
  Table: check_constraint_routine_usage
  Table: column_column_usage
  Table: check_constraints
  Table: constraint_table_usage
  Table: column_domain_usage
  Table: columns
  Table: column_privileges
  Table: constraint_column_usage
  Table: domain_udt_usage
  Table: domains
  Table: enabled_roles
  Table: key_column_usage
  Table: parameters
  Table: table_privileges
  Table: referential_constraints
  Table: sql_features
  Table: role_column_grants
  Table: routine_column_usage
  Table: routine_privileges
  Table: role_routine_grants
  Table: routine_routine_usage
  Table: sql_implementation_info
  Table: routine_sequence_usage
  Table: routine_table_usage


In [30]:
import pandas as pd

# Define the column names for your DataFrame
column_names = ["id", "user", "event", "timestamp", "text", "action_name", "metadata"]

# Create the DataFrame
df = pd.DataFrame(rows, columns=column_names)
import json
from pandas import json_normalize

# Convert the JSON strings in the 'metadata' column to dictionaries
df['metadata'] = df['metadata'].apply(json.loads)

# Unnest the 'metadata' column into a separate DataFrame
metadata_df = json_normalize(df['metadata'])

# Add a suffix to the column names in metadata_df
metadata_df.columns = [str(col) + '_metadata' for col in metadata_df.columns]

# Join the DataFrames
df2 = df.drop(columns=['metadata']).join(metadata_df)

df.head()

,id,user,event,timestamp,text,action_name,metadata
0,1,user,action,1.702754e+09,None,action_session_start,"{'event': 'action', 'timestamp': 1702754175.57..."
1,2,user,session_started,1.702754e+09,None,None,"{'event': 'session_started', 'timestamp': 1702..."
2,3,user,action,1.702754e+09,None,action_listen,"{'event': 'action', 'timestamp': 1702754175.57..."
3,4,user,user,1.702754e+09,greet,None,"{'event': 'user', 'timestamp': 1702754176.9069..."
4,5,user,user_featurization,1.702754e+09,None,None,"{'event': 'user_featurization', 'timestamp': 1..."


In [34]:
df.loc[df['action_name']=='utter_happy','metadata']

10    {'event': 'action', 'timestamp': 1702754194.76...
Name: metadata, dtype: object

In [35]:
metadata_df.head()

,event_metadata,timestamp_metadata,name_metadata,policy_metadata,confidence_metadata,action_text_metadata,hide_rule_turn_metadata,metadata.model_id_metadata,metadata.assistant_id_metadata,text_metadata,...,parse_data.response_selector.default.response.utter_action_metadata,parse_data.response_selector.default.ranking_metadata,use_text_for_featurization_metadata,metadata.utter_action_metadata,data.elements_metadata,data.quick_replies_metadata,data.buttons_metadata,data.attachment_metadata,data.image_metadata,data.custom_metadata
0,action,1.702754e+09,action_session_start,None,1.0,NaN,False,f103e03f3fba4252aaa29cf620dcf365,20231203-154440-odious-bollard,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,session_started,1.702754e+09,NaN,NaN,NaN,NaN,NaN,f103e03f3fba4252aaa29cf620dcf365,20231203-154440-odious-bollard,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,action,1.702754e+09,action_listen,None,NaN,NaN,False,f103e03f3fba4252aaa29cf620dcf365,20231203-154440-odious-bollard,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,user,1.702754e+09,NaN,NaN,NaN,NaN,NaN,f103e03f3fba4252aaa29cf620dcf365,20231203-154440-odious-bollard,hi,...,utter_None,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,user_featurization,1.702754e+09,NaN,NaN,NaN,NaN,NaN,f103e03f3fba4252aaa29cf620dcf365,20231203-154440-odious-bollard,NaN,...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
